In [157]:
def analyse_exploration():
    global EXPLORATION
    input_samples = []
    output_samples = []
    games = []
    for i in range(100):
        game = generate_random_game()
        games.append(game)
        input_samples.append(generate_input(*game))
        output_samples.append(calc_correct_output_sample(*game))        
    
    prev_exp = EXPLORATION
    for i in range(1, 20):   
        EXPLORATION = i    
        correct = []

        for s in range(100, 500, 100):
            correct.append(0)
            for g in range(len(games)): 
                set_env(*games[g])
                root = State(1, env.get_state(), env.current_player)
                _, output = mcts_game_step(root, s)

                correct_output = output_samples[g]
                correct[-1] += correct_output[np.argmax(output)] == correct_output[np.argmax(correct_output)]
        print(i, np.array(correct) / len(games))


    EXPLORATION = prev_exp

In [315]:
def eval_mcts(model, eval_cache_input, eval_cache_output):
    
    for s in range(1, 20, 2):
        correct = 0
        for i in range(len(eval_cache_input)):

            outputs = np.zeros((s, 32))
            for g in range(s):
                game = generate_game_from_obs(eval_cache_input[i])
                set_env(*game)
                root = State(1, env.get_state(), env.current_player)
                _, p = mcts_game_step(root)

                set_env(*game)
                for n, card in enumerate(env.players[env.current_player].hand_cards):
                    outputs[g][card.id] = p[n]

            outputs = np.mean(outputs, axis=0)
            correct_output = eval_cache_output[i]

            correct += correct_output[np.argmax(outputs)] == correct_output[np.argmax(correct_output)]
            #if correct_output[np.argmax(output)] != correct_output[np.argmax(correct_output)]:
             #   print(i, correct_output[np.argmax(output)], correct_output[np.argmax(correct_output)])

        print(s, correct / len(eval_cache_input))


In [264]:
generate_game_from_obs(eval_cache_input[0])[0][1].value

<Value.SAU: 7>

In [ ]:
eval_mcts(model, eval_cache_input, eval_cache_output)

1 0.79
3 0.75
5 0.78
7 0.78


In [166]:
show_mcts([
    card(Color.HERZ, Value.ZEHN),
    card(Color.GRUEN, Value.SAU),
    card(Color.SCHELLN, Value.SIEBEN),
    card(Color.SCHELLN, Value.NEUN),
    card(Color.EICHEL, Value.UNTER)
], None, 0, 0, [
    card(Color.EICHEL, Value.ZEHN),
    card(Color.HERZ, Value.SAU),
    card(Color.SCHELLN, Value.OBER),
    card(Color.SCHELLN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG)
])

FigureCanvasNbAgg()

array([0.09027824, 0.25778008, 0.30616403, 0.17416755, 0.1716101 ])

In [185]:
%lprun -f mcts_sample show_mcts([card(Color.HERZ, Value.ZEHN),card(Color.GRUEN, Value.SAU),card(Color.SCHELLN, Value.SIEBEN),card(Color.SCHELLN, Value.NEUN),card(Color.EICHEL, Value.UNTER)])


Timer unit: 1e-06 s

Total time: 0.438474 s
File: <ipython-input-133-7f0ddb0fbb17>
Function: mcts_sample at line 39

Line #      Hits         Time  Per Hit   % Time  Line Contents
    39                                           def mcts_sample(state, use_model=True):
    40                                               
    41       385        850.0      2.2      0.2      if state.is_leaf_node():
    42       100        106.0      1.1      0.0          if state.end_v != 0:
    43                                                       v = state.end_v
    44                                                       player = -1
    45                                                   else:
    46       100        743.0      7.4      0.2              env.set_state(state.env_state)
    47                                           
    48       100        101.0      1.0      0.0              if use_model:
    49       100     357269.0   3572.7     81.5                  p, v = model.predict_singl

In [144]:
model.predict_single(generate_input([
    card(Color.HERZ, Value.ZEHN),
    card(Color.GRUEN, Value.SAU),
    card(Color.SCHELLN, Value.NEUN),
    card(Color.EICHEL, Value.UNTER)
], None, 1, 0, [
    card(Color.HERZ, Value.SAU),
    card(Color.SCHELLN, Value.OBER),
    card(Color.SCHELLN, Value.KOENIG),
    card(Color.EICHEL, Value.KOENIG)
]))

[array([0.00846176, 0.00365888, 0.00696691, 0.32488713, 0.00436447,
        0.00803394, 0.00630486, 0.00651327, 0.25214696, 0.00646617,
        0.0053455 , 0.00681768, 0.00575565, 0.00709505, 0.00533196,
        0.00393214, 0.00878035, 0.00780082, 0.00432327, 0.00539902,
        0.2030162 , 0.00293943, 0.00468038, 0.01278391, 0.01050032,
        0.00696515, 0.00696844, 0.01051198, 0.008347  , 0.3725077 ,
        0.00666502, 0.00507753], dtype=float32),
 array([0.7587035], dtype=float32)]